In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pysal as ps
import geopandas as gpd

plt.rcParams['figure.figsize'] = [15, 8]
pd.set_option('display.max_columns', None)

In [4]:
os.listdir('../data')

['df.xlsx',
 'holidays_events.csv',
 'items.csv',
 'oil.csv',
 'sample_submission.csv',
 'stores.csv',
 'test.csv',
 'train+oil.xlsx',
 'train.csv',
 'transactions.csv']

In [6]:
train_df = pd.read_csv('../data/train.csv')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
train_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [7]:
stores_df = pd.read_csv('../data/stores.csv')

In [14]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [81]:
# find location of target store (store_nbr == 44)
stores_df[stores_df['store_nbr'] == 44]

,store_nbr,city,state,type,cluster
43,44,Quito,Pichincha,A,5


In [82]:
# find other stores in the same cluster/city
tmp = stores_df[(stores_df['cluster'] == 5) | (stores_df['city'] == 'Quito')]
tmp

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15
16,17,Quito,Pichincha,C,12


In [83]:
neighbours = tmp['store_nbr'].tolist()
neighbours.remove(44)

In [84]:
neighbours

[1, 2, 3, 4, 6, 7, 8, 9, 10, 17, 18, 20, 45, 46, 47, 48, 49]

In [85]:
neighbours_df = []

# check if neighbours sell the same item as the target store (1503844)
tmp_df = train_df[(train_df['store_nbr'].isin(neighbours)) & (train_df['item_nbr'] == 1503844)]

In [86]:
tmp_df.shape

(12264, 6)

In [87]:
tmp_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
16326059,16326059,2014-01-02,1,1503844,205.527,NaN
16327810,16327810,2014-01-02,2,1503844,281.905,NaN
16329733,16329733,2014-01-02,3,1503844,615.122,NaN
16331355,16331355,2014-01-02,4,1503844,237.730,NaN
16334776,16334776,2014-01-02,6,1503844,287.284,NaN


In [88]:
tmp_df.reset_index(inplace = True, drop = True)

In [89]:
tmp_df = tmp_df.groupby(['store_nbr', 'date'], as_index=False).max()
tmp_df.head()

,store_nbr,date,id,item_nbr,unit_sales,onpromotion
0,1,2014-01-02,16326059,1503844,205.527,NaN
1,1,2014-01-03,16392837,1503844,193.054,NaN
2,1,2014-01-04,16457927,1503844,153.865,NaN
3,1,2014-01-05,16526561,1503844,44.622,NaN
4,1,2014-01-06,16594508,1503844,113.703,NaN


In [90]:
tmp_df.tail()

,store_nbr,date,id,item_nbr,unit_sales,onpromotion
12259,49,2017-08-11,125068699,1503844,468.598,False
12260,49,2017-08-12,125173925,1503844,507.750,False
12261,49,2017-08-13,125279686,1503844,624.804,False
12262,49,2017-08-14,125383380,1503844,622.435,False
12263,49,2017-08-15,125486176,1503844,426.359,False


In [157]:
df = train_df[(train_df['store_nbr'] == 44) & (train_df['item_nbr'] == 1503844)]
exog = []
for n in neighbours:
    t = tmp_df[tmp_df['store_nbr'] == n][['unit_sales', 'date']]
    df = pd.merge(df, t, how = 'outer', on = 'date', suffixes = ('', str(n)))
    exog.append('unit_sales' + str(n))

df = df.fillna(0)

In [158]:
df.head(50)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,unit_sales1,unit_sales2,unit_sales3,unit_sales4,unit_sales6,unit_sales7,unit_sales8,unit_sales9,unit_sales10,unit_sales17,unit_sales18,unit_sales20,unit_sales45,unit_sales46,unit_sales47,unit_sales48,unit_sales49
0,16377457,2014-01-02,44,1503844,966.1490,0,205.5270,281.9050,615.1220,237.7300,287.2840,497.2570,500.7700,0.0,0.0,0.0,0.0,0.0,685.3920,212.1760,643.9320,198.0140,399.1760
1,16442863,2014-01-03,44,1503844,663.4050,0,193.0540,203.4590,577.9060,141.4590,231.1620,480.9320,275.4320,0.0,0.0,0.0,0.0,0.0,473.4860,103.8650,432.2570,112.8110,352.1350
2,16511061,2014-01-04,44,1503844,1075.0000,0,153.8650,219.7700,804.3920,222.1490,280.8240,463.3780,570.1890,0.0,0.0,0.0,0.0,0.0,790.0680,231.8510,566.1220,190.6080,593.2430
3,16578789,2014-01-05,44,1503844,892.6080,0,44.6220,294.9590,506.9050,136.7430,182.7030,508.4190,339.0680,0.0,0.0,0.0,0.0,0.0,407.3240,183.6760,449.4050,257.2300,577.9190
4,16642520,2014-01-06,44,1503844,261.5540,0,113.7030,62.5000,153.1620,114.8780,92.7700,480.7430,261.9590,0.0,0.0,0.0,0.0,0.0,366.5680,51.6350,103.4590,133.0680,409.2840
5,16705308,2014-01-07,44,1503844,646.7160,0,170.6890,221.0950,452.2300,139.4190,223.2030,390.6080,462.5410,0.0,0.0,0.0,0.0,0.0,462.0000,99.4320,389.2160,65.6490,347.3920
6,16768213,2014-01-08,44,1503844,1165.2097,0,224.1935,264.3710,812.0968,240.1290,228.5000,512.8226,608.5161,0.0,0.0,0.0,0.0,0.0,658.5645,161.7742,549.8548,126.0968,495.3710
7,16830673,2014-01-09,44,1503844,620.6760,0,134.4460,169.5810,534.2020,125.9320,146.2570,346.0540,345.0410,0.0,0.0,0.0,0.0,0.0,351.0270,93.7840,283.1080,145.6220,361.6220
8,16892999,2014-01-10,44,1503844,602.2430,0,147.8780,152.0000,432.8650,132.8780,164.0810,345.8650,340.8380,0.0,0.0,0.0,0.0,0.0,450.4190,113.8650,307.1350,78.6620,413.3110
9,16960548,2014-01-11,44,1503844,1090.4050,0,187.2840,233.1760,599.1620,188.4460,203.8380,381.2570,550.8780,0.0,0.0,0.0,0.0,0.0,579.0950,180.9460,539.3240,149.4460,481.2970


In [159]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [160]:
train_ratio = 0.7
train, test = df.iloc[:round(train_ratio * df.shape[0]), :], df.iloc[round(train_ratio * df.shape[0]):, :]
train.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)

print(train.shape)
print(test.shape)

(711, 23)
(305, 23)


In [128]:
p, d, q, s = 2, 1, 2, 7
model  = SARIMAX(endog = train['unit_sales'], exog = train[exog], order = (p, d, q), seasonal_order = (p, d, q, s))
model_fit = model.fit(disp = 0)
model_fit.summary()

D:\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                        unit_sales   No. Observations:                  711
Model:             SARIMAX(2, 1, 2)x(2, 1, 2, 7)   Log Likelihood               -4186.257
Date:                           Fri, 04 Oct 2019   AIC                           8424.515
Time:                                   15:49:38   BIC                           8542.954
Sample:                                        0   HQIC                          8470.290
                                           - 711                                         
Covariance Type:                             opg                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
unit_sales1     -0.0085      0.108     -0.078      0.938      -0.221       0.204
unit_sales2      0.1801      0.116      1.558      0.119      -0.046       0.407
unit_sales3      0.1584      0.050      3.187      0.001       0.061       0.256
unit_sales4      0.5930      0.145      4.092      0.000       0.309       0.877
unit_sales6      0.2101      0.097      2.163      0.031       0.020       0.400
unit_sales7      0.0016      0.065      0.025      0.980      -0.126       0.129
unit_sales8      0.4778      0.052      9.264      0.000       0.377       0.579
unit_sales9      3.3602      9.803      0.343      0.732     -15.853      22.573
unit_sales10     4.3964      9.218      0.477      0.633     -13.671      22.464
unit_sales17 -6.724e-15   4925.569  -1.37e-18      1.000   -9653.938    9653.938
unit_sales18     4.9788      1.225      4.064      0.000       2.578       7.380
unit_sales20     5.3245      3.696      1.441      0.150      -1.919      12.568
unit_sales45     0.2292      0.059      3.888      0.000       0.114       0.345
unit_sales46     0.3798      0.138      2.755      0.006       0.110       0.650
unit_sales47     0.2980      0.068      4.376      0.000       0.165       0.432
unit_sales48     0.1738      0.134      1.294      0.196      -0.089       0.437
unit_sales49     0.0967      0.051      1.899      0.058      -0.003       0.196
ar.L1           -0.9259     13.750     -0.067      0.946     -27.875      26.023
ar.L2            0.0738      1.019      0.072      0.942      -1.924       2.072
ma.L1           -0.0003     13.719  -1.95e-05      1.000     -26.890      26.889
ma.L2           -0.9997     13.748     -0.073      0.942     -27.946      25.946
ar.S.L7         -0.8989      0.129     -6.956      0.000      -1.152      -0.646
ar.S.L14         0.0119      0.049      0.245      0.806      -0.083       0.107
ma.S.L7          0.0699      0.117      0.597      0.551      -0.160       0.300
ma.S.L14        -0.8083      0.107     -7.541      0.000      -1.018      -0.598
sigma2        8411.9863      0.677   1.24e+04      0.000    8410.659    8413.314
===================================================================================
Ljung-Box (Q):                       41.60   Jarque-Bera (JB):               288.08
Prob(Q):                              0.40   Prob(JB):                         0.00
Heteroskedasticity (H):               0.92   Skew:                             0.13
Prob(H) (two-sided):                  0.51   Kurtosis:                         6.13
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 1.98e+24. Standard errors may be unstable.
"""

In [173]:
# remove those that are not significant
# exog.remove('unit_sales1')
# exog.remove('unit_sales3')
# exog.remove('unit_sales4')
# exog.remove('unit_sales8')
# exog.remove('unit_sales9')
# exog.remove('unit_sales10')
# exog.remove('unit_sales17')
# exog.remove('unit_sales18')
# exog.remove('unit_sales20')
# exog.remove('unit_sales45')
# exog.remove('unit_sales46')
# exog.remove('unit_sales47')

p, d, q, s = 2, 1, 2, 7
model  = SARIMAX(endog = train['unit_sales'], exog = train[exog], order = (p, d, q), seasonal_order = (p, d, q, s), enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit(disp = 0)
model_fit.summary()

D:\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                        unit_sales   No. Observations:                  711
Model:             SARIMAX(2, 1, 2)x(2, 1, 2, 7)   Log Likelihood               -4249.244
Date:                           Fri, 04 Oct 2019   AIC                           8526.489
Time:                                   16:52:17   BIC                           8589.921
Sample:                                        0   HQIC                          8551.032
                                           - 711                                         
Covariance Type:                             opg                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
unit_sales2      1.0767      0.208      5.186      0.000       0.670       1.484
unit_sales6      0.6251      0.144      4.328      0.000       0.342       0.908
unit_sales7      0.1350      0.122      1.103      0.270      -0.105       0.375
unit_sales48     0.7289      0.245      2.981      0.003       0.250       1.208
unit_sales49     0.4665      0.090      5.200      0.000       0.291       0.642
ar.L1            0.7232      0.936      0.772      0.440      -1.112       2.558
ar.L2           -0.0608      0.079     -0.774      0.439      -0.215       0.093
ma.L1           -1.6635      0.937     -1.776      0.076      -3.499       0.172
ma.L2            0.6646      0.936      0.710      0.478      -1.170       2.499
ar.S.L7         -0.4801      0.631     -0.761      0.447      -1.717       0.757
ar.S.L14        -0.0701      0.100     -0.701      0.483      -0.266       0.126
ma.S.L7         -0.2380      0.641     -0.372      0.710      -1.493       1.017
ma.S.L14        -0.2960      0.460     -0.644      0.519      -1.197       0.605
sigma2        2.125e+04   1408.199     15.087      0.000    1.85e+04     2.4e+04
===================================================================================
Ljung-Box (Q):                       44.38   Jarque-Bera (JB):               180.82
Prob(Q):                              0.29   Prob(JB):                         0.00
Heteroskedasticity (H):               0.77   Skew:                             0.08
Prob(H) (two-sided):                  0.05   Kurtosis:                         5.51
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [175]:
# remove those that are not significant
# exog.remove('unit_sales2')
# exog.remove('unit_sales6')
# exog.remove('unit_sales48')
# exog.remove('unit_sales49')

p, d, q, s = 2, 1, 2, 7
model  = SARIMAX(endog = train['unit_sales'], exog = train[exog], order = (p, d, q), seasonal_order = (p, d, q, s), enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit(disp = 0)
model_fit.summary()

D:\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                        unit_sales   No. Observations:                  711
Model:             SARIMAX(2, 1, 2)x(2, 1, 2, 7)   Log Likelihood               -4399.419
Date:                           Fri, 04 Oct 2019   AIC                           8818.838
Time:                                   16:55:28   BIC                           8864.147
Sample:                                        0   HQIC                          8836.369
                                           - 711                                         
Covariance Type:                             opg                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
unit_sales7     0.9112      0.098      9.322      0.000       0.720       1.103
ar.L1           0.7937      0.191      4.155      0.000       0.419       1.168
ar.L2          -0.2205      0.054     -4.121      0.000      -0.325      -0.116
ma.L1          -1.6797      0.193     -8.697      0.000      -2.058      -1.301
ma.L2           0.6805      0.194      3.513      0.000       0.301       1.060
ar.S.L7        -0.4304      0.543     -0.793      0.428      -1.494       0.634
ar.S.L14        0.0297      0.122      0.243      0.808      -0.210       0.269
ma.S.L7        -0.1604      0.545     -0.295      0.768      -1.228       0.907
ma.S.L14       -0.2595      0.291     -0.892      0.372      -0.830       0.311
sigma2        3.12e+04   1944.558     16.044      0.000    2.74e+04     3.5e+04
===================================================================================
Ljung-Box (Q):                       62.93   Jarque-Bera (JB):               307.97
Prob(Q):                              0.01   Prob(JB):                         0.00
Heteroskedasticity (H):               0.67   Skew:                             0.44
Prob(H) (two-sided):                  0.00   Kurtosis:                         6.16
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [177]:
# remove those that are not significant
# exog.remove('unit_sales7')

p, d, q, s = 2, 1, 2, 7
model  = SARIMAX(endog = train['unit_sales'], order = (p, d, q), seasonal_order = (p, d, q, s), enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit(disp = 0)
model_fit.summary()

D:\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                        unit_sales   No. Observations:                  711
Model:             SARIMAX(2, 1, 2)x(2, 1, 2, 7)   Log Likelihood               -4466.056
Date:                           Fri, 04 Oct 2019   AIC                           8950.112
Time:                                   16:56:06   BIC                           8990.890
Sample:                                        0   HQIC                          8965.890
                                           - 711                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8901      0.188      4.740      0.000       0.522       1.258
ar.L2         -0.2319      0.048     -4.790      0.000      -0.327      -0.137
ma.L1         -1.7264      0.184     -9.366      0.000      -2.088      -1.365
ma.L2          0.7268      0.185      3.936      0.000       0.365       1.089
ar.S.L7       -0.4447      0.549     -0.810      0.418      -1.521       0.631
ar.S.L14       0.0305      0.124      0.247      0.805      -0.212       0.273
ma.S.L7       -0.1280      0.550     -0.233      0.816      -1.207       0.951
ma.S.L14      -0.2729      0.296     -0.921      0.357      -0.854       0.308
sigma2      3.763e+04   2356.537     15.968      0.000     3.3e+04    4.22e+04
===================================================================================
Ljung-Box (Q):                       72.75   Jarque-Bera (JB):               505.02
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.68   Skew:                             0.24
Prob(H) (two-sided):                  0.00   Kurtosis:                         7.17
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [180]:
endog = train['unit_sales']
predictions = []

for i in range(len(test)):
    model  = SARIMAX(endog = endog, order = (p, d, q), seasonal_order = (p, d, q, s), enforce_stationarity=False, enforce_invertibility=False)
    model_fit = model.fit(disp = 0)
    output = model_fit.predict()
    yhat = output[0]
    predictions.append(yhat)
    obs = test.iloc[i][['unit_sales']]
    endog.append(obs)
    
    print('predicted = {}, expected = {}'.format(yhat, obs['unit_sales']))